In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import scipy
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os
import creep_event_picker as cep
import obspy
from matplotlib.dates import num2date
import matplotlib.dates as mdates
%matplotlib qt

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
def check_dir(path):
    isExist = os.path.exists(path)
    if not isExist:
        # Create a new directory because it does not exist 
        os.makedirs(path, exist_ok=True)

Import creepmeter database

In [3]:
Creepmeter_dataframe = pd.read_csv('../../Data/creepmeter_lookup.csv',index_col=0)
#Creepmeter_dataframe = pd.read_excel('../../Data/creepmeter_lookup_and_processing_progress.xlsx',index_col=0)

Import data

In [4]:
creepmeter ='SRI1'

In [5]:
creepmeter_info = Creepmeter_dataframe.loc[Creepmeter_dataframe.index[Creepmeter_dataframe.Creepmeter_abbrv==creepmeter]]

In [6]:
creepmeter_info

,Network,Creepmeter_full_name,Creepmeter_abbrv,File_code,Start Time,End Time,"Sampling rate, Hz","Sampling rate, mins",Latitude,Longitude,Gittins_and_Hawthorne_2022
56,East_Anatolian_Fault,Srivice,SRI1,sri_0,2020-02-05T11:48:00.000000Z,2023-12-02T11:19:00.000000Z,0.016667,1,38.388,39.1873,False


In [9]:

plt.close('all')
for i in range(len(creepmeter_info)):
    path = "../../Data/{p}/SAC/{q}.SAC".format(p=creepmeter_info.Network.iloc[i],q =creepmeter_info.File_code.iloc[i])
    path_save = "../../Data/{p}/Picks/".format(p=creepmeter_info.Network.iloc[i])
    check_dir(path_save)
    print(path)
    st = obspy.read(path)
    tr = st[0]
    tr.interpolate(sampling_rate=tr.stats.sampling_rate)
    tm = tr.times("utcdatetime")
    creep = tr.data
    sample_rate =  round(tr.stats.delta)/60
    tm_dt = []
    for w in range(len(tm)):
        tm_new = tm[w].datetime
        tm_dt.append(tm_new)
    print('now interpolate')
    tm_int, creep_int, upsampled = cep.interpolate(tm_dt,creep,sample_rate,5)

    time_in_mins = int(4*10080/sample_rate)#10080
    if creepmeter_info.Gittins_and_Hawthorne_2022.iloc[i] == 'True':
        df_picks_pre_picked = pd.read_csv("../../Data/{p}/Picks/{q}_picks.csv".format(p=creepmeter_info.Network.iloc[0],q =creepmeter_info.File_code.iloc[0]),index_col=0)
        df_picks = pd.read_csv("../../Data/{p}/Picks/{q}_picks.csv".format(p=creepmeter_info.Network.iloc[0],q =creepmeter_info.File_code.iloc[i]),index_col=0)
    else:
        df_picks=pd.DataFrame(columns=['ST'])
    for k in range(round(len(tm)/time_in_mins)-1):
        fig=plt.figure()
        ax = plt.subplot(1,1,1)
        plt.plot(tm_int[k*time_in_mins:(k+1)*time_in_mins],creep_int[k*time_in_mins:(k+1)*time_in_mins]-creep_int[k*time_in_mins:(k+1)*time_in_mins][0])
        bottom,top = ax.get_ylim()
        try:
            for q in range(len(df_picks)):
                plt.plot([pd.to_datetime(df_picks.ST.iloc[q]),pd.to_datetime(df_picks.ST.iloc[q])],[bottom,top],color='orange')
        except:
            print('woo')
        ax.set_xlim([tm_int[k*time_in_mins],tm_int[(k+1)*time_in_mins]])
        ax.set_ylim([bottom,top])
        scale = 1.1
        zp = cep.ZoomPan()
        figZoom = zp.zoom_factory(ax, base_scale = scale)
        figPan = zp.pan_factory(ax)
        s = 1

        fig.set_size_inches(11,11)
        plt.show()
        pts = plt.ginput(100,timeout = -1)
        plt.close('all')
        for sublist in pts:

            df = pd.DataFrame({'ST':[sublist[0]]})
            df_picks = pd.concat([df_picks,df])
    print(len(df_picks))
    
    if len(df_picks) >0:
        for r in range(len(df_picks)):

            try:
                time_convert = mdates.num2date(df_picks.ST.iloc[r])
                s3 = pd.Series(time_convert).dt.round("1min")
                df_picks.ST.iloc[r] = s3.iloc[0].replace(tzinfo=None)   
            except TypeError:
                dummy=10
        
        df_picks['ST'] = df_picks['ST'].astype(str).str[0:19]
        df_picks.ST = pd.to_datetime(df_picks.ST)
        df_picks.sort_values(by='ST',inplace=True)
        #df_picks.reset_index(inplace=True)
        df_picks.to_csv("../../Data/{p}/Picks/{q}_picks.csv".format(p=creepmeter_info.Network.iloc[i],q =creepmeter_info.File_code.iloc[i]))

        
        plt.close('all')
        df_picks_end=pd.DataFrame(columns=['ET'])
        for j in range(len(df_picks)):
            boolarr = (tm_int>=pd.to_datetime(df_picks.ST.iloc[j]).replace(tzinfo=None)-dt.timedelta(hours=2)) \
                & (tm_int<=pd.to_datetime(df_picks.ST.iloc[j]).replace(tzinfo=None)+dt.timedelta(hours=72))
            tm_event = tm_int[boolarr]
            creep_event = creep_int[boolarr]
            
                                


            fig=plt.figure()
            ax = plt.subplot(1,1,1)
            if len(creep_event) >= 1:
                plt.plot(tm_event,creep_event-creep_event[0])
                bottom,top = ax.get_ylim()
                plt.plot([pd.to_datetime(df_picks.ST.iloc[j]),pd.to_datetime(df_picks.ST.iloc[j])],[bottom,top])
                if j < len(df_picks)-1:
                    plt.plot([pd.to_datetime(df_picks.ST.iloc[j+1]),pd.to_datetime(df_picks.ST.iloc[j+1])],[bottom,top])
                ax.set_ylim(bottom,top)   
                ax.set_xlim(tm_event[0],tm_event[-1])
                scale = 1.1
                zp = cep.ZoomPan()
                figZoom = zp.zoom_factory(ax, base_scale = scale)
                figPan = zp.pan_factory(ax)
                s = 1
                fig.set_size_inches(11,11)
                plt.title('{k}/{p}'.format(k=j+1,p=len(df_picks)))
                plt.show()
                pts = plt.ginput(1,timeout = -1)
                plt.close()
                for sublist in pts:
                    df = pd.DataFrame({'ET':[sublist[0]]})
                    df_picks_end = pd.concat([df_picks_end,df]) 
                    #print(df_picks_end)
            else:
                df = pd.DataFrame({'ET':[dt.datetime(2100,1,1,0,0,0)]})
                df_picks_end = pd.concat([df_picks_end,df])

        for r in range(len(df_picks)):
            time_convert = mdates.num2date(float(df_picks_end.ET.iloc[r]))
            s3 = pd.Series(time_convert).dt.round("1min")
            df_picks_end.ET.iloc[r] = s3.iloc[0].replace(tzinfo=None)  
        
        #df_picks_end.ET= mdates.num2date(df_picks_end.ET)
        df_picks_end.reset_index(inplace=True)
        df_picks_end.drop(columns=['index'],inplace=True)

        

        df_picks['ET'] = np.ones(len(df_picks))
        for r in range(len(df_picks)):
            df_picks.ET.iloc[r]=df_picks_end.ET.iloc[r]

        df_picks.to_csv("../../Data/{p}/Picks/{q}_picks.csv".format(p=creepmeter_info.Network.iloc[i],\
                                                                       q =creepmeter_info.File_code.iloc[i]))

../../Data/East_Anatolian_Fault/SAC/sri_0.SAC
now interpolate


/Users/dagi9522/Library/CloudStorage/OneDrive-UCB-O365/Documents/GitHub/Creep_event_detection_scripts/creep_event_picker.py:638: FutureWarning: DataFrame.interpolate with method=ffill is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  interpolated = upsampled.interpolate(method='ffill') #interpolate the dataset to get a continious record evenly spaced at 10 mins
/var/folders/56/47k8q3q1449cq8kwzkz5r8ch0000gr/T/ipykernel_54612/2809728349.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_picks = pd.concat([df_picks,df])


1


/var/folders/56/47k8q3q1449cq8kwzkz5r8ch0000gr/T/ipykernel_54612/2809728349.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2020-02-10 20:41:00' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_picks.ST.iloc[r] = s3.iloc[0].replace(tzinfo=None)
/var/folders/56/47k8q3q1449cq8kwzkz5r8ch0000gr/T/ipykernel_54612/2809728349.py:104: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_picks_end = pd.concat([df_picks_end,df])
/var/folders/56/47k8q3q1449cq8kwzkz5r8ch0000gr/T/ipykernel_54612/2809728349.py:113: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2020-02